In [71]:
# 데이터 처리
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# 선형 회귀분석모델 formula(y ~ X1 + X2 + ...)
import statsmodels.formula.api as smf
# 회귀분석의 정규성 확인
from statsmodels.api import qqplot

# 실행결과 경고메시지 출력 제외
import warnings

warnings.filterwarnings('ignore')

# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

In [72]:
# 45 Page
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 선형 회귀모델 formula(y ~ X1 + X2 + ...)
import statsmodels.formula.api as smf
# 잔차의 정규성 확인 :qqplot, 회귀분석의 상수항 추가 : add_constant
from statsmodels.api import qqplot, add_constant
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 학습용/평가용 데이터 분리:train, test
from sklearn.model_selection import train_test_split
# 선형회귀모형
from sklearn.linear_model import LinearRegression
# 변수 선택법(후진제거법)
from sklearn.feature_selection import RFE
# Scale 변환->표준화 회귀계수 산출
from sklearn.preprocessing import StandardScaler
# 평가함수
from sklearn.metrics import r2_score

In [73]:
df = pd.read_csv("./CU_찐최종.csv")
df.columns

Index(['idUser', 'Gender', 'Age', 'AgeGroup', 'Recency', 'Frequency',
       'Monetary', 'RFM_Score', '2022_Cus_Grade1', '2022_Cus_Grade2',
       '2023_Cus_Grade1', '2023_Cus_Grade2', '2022 상반기_고객등급', '2022 하반기_고객등급',
       '2023 상반기_고객등급', '2023 하반기_고객등급', 'Visit_Score', 'SalesDate', 'SalesDT',
       'Year', 'Month', 'Day', 'WeekdayName', 'Hour', 'ItemId', 'idPOS',
       'ItemNameLarge', 'ItemNameMiddle', 'ItemName', 'Item_CatORMaker',
       'ImageUrl', 'Price', 'Sales_Day', 'Sales_Month', 'Sales_Year',
       'PayUnit_Equal_Day', 'PayUnit_Diff_Day', 'PayUnit_Equal_Month',
       'PayUnit_Diff_Month', 'PayUnit_Equal_Year', 'PayUnit_Diff_Year',
       'TempAvg', 'TempMin', 'hhmiTempMin', 'TempMax', 'hhmiTempMax',
       'Temp_Var', 'hourRainfall', 'RainfallAmount', 'HumidAvg', 'SnowloadMax',
       'hhmiSnowloadMax', 'Discomfort_Grade'],
      dtype='object')

In [74]:
df['SalesDate']

0         2022-01-02
1         2022-01-02
2         2022-01-06
3         2022-01-09
4         2023-04-13
             ...    
498002    2023-01-07
498003    2023-01-08
498004    2023-01-08
498005    2023-01-13
498006    2023-01-13
Name: SalesDate, Length: 498007, dtype: object

In [75]:
# 중복된 모든 행 확인하기
duplicates = df[df.duplicated()]

print("중복 데이터 개수:")
print(len(duplicates))

중복 데이터 개수:
0


In [76]:
df1 = df.copy()

In [77]:
# 'SalesDate' 칼럼을 datetime 형태로 변환
df1['SalesDate'] = pd.to_datetime(df1['SalesDate'])

# 월 정보 추출
df1['Month'] = df1['SalesDate'].dt.to_period('M')

# idPOS, Month, Gender별로 중복 제거
unique_visits = df1.drop_duplicates(subset=['idPOS', 'Month', 'Gender'])

# 월별, 성별로 그룹화하여 각각의 고객 수 계산
monthly_gender_visits = unique_visits.groupby(['Month', 'Gender'])['idPOS'].count().unstack(fill_value=0)

# monthly_gender_visits 데이터프레임에서 남성 및 여성 고객 수를 df1에 매핑하기 위해
# df1의 'Month' 칼럼을 기준으로 join을 수행합니다.
# 먼저 monthly_gender_visits의 인덱스를 컬럼으로 리셋합니다.
monthly_gender_visits_reset = monthly_gender_visits.reset_index()

# 'Month' 칼럼을 기준으로 df1과 monthly_gender_visits_reset를 merge합니다.
# 이 때, left_on과 right_on 파라미터를 사용하여 'Month' 칼럼을 기준으로 합니다.
# 'how' 파라미터는 'left'로 설정하여 df1의 모든 행이 유지되도록 합니다.
df1 = pd.merge(df1, monthly_gender_visits_reset, how='left', left_on='Month', right_on='Month')

# 결과적으로 merge된 데이터프레임에는 '남성' 및 '여성' 칼럼이 추가됩니다.
# 이 칼럼들의 이름을 '월별남자고객수', '월별여자고객수'로 변경합니다.
df1.rename(columns={'남성': '월별남자고객수', '여성': '월별여자고객수'}, inplace=True)

# 최종 데이터프레임 확인
print(df1.head())


    idUser Gender  Age AgeGroup  Recency  Frequency  Monetary  RFM_Score  \
0  U100001     남성   43      40대    1.000      0.000     0.000      0.000   
1  U100001     남성   43      40대    1.000      0.000     0.000      0.000   
2  U100001     남성   43      40대    1.000      0.000     0.000      0.000   
3  U100001     남성   43      40대    1.000      0.000     0.000      0.000   
4  U100277     여성   33      30대    0.001      0.298     0.218      0.616   

  2022_Cus_Grade1 2022_Cus_Grade2  ... hhmiTempMax Temp_Var hourRainfall  \
0             브론즈             NaN  ...        1551      8.2         4.17   
1             브론즈             NaN  ...        1551      8.2         4.17   
2             브론즈             NaN  ...        1604      9.2         0.00   
3             브론즈             NaN  ...        1522      6.8         0.00   
4             NaN             NaN  ...        1512     12.1         0.25   

  RainfallAmount HumidAvg SnowloadMax hhmiSnowloadMax Discomfort_Grade  \
0           

In [78]:
monthly_gender_visits.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 24 entries, 2022-01 to 2023-12
Freq: M
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   남성      24 non-null     int64
 1   여성      24 non-null     int64
dtypes: int64(2)
memory usage: 576.0 bytes


In [80]:
df1

idUser                                                           U100001
Gender                                                                남성
Age                                                                   43
AgeGroup                                                             40대
Recency                                                              1.0
Frequency                                                            0.0
Monetary                                                             0.0
RFM_Score                                                            0.0
2022_Cus_Grade1                                                      브론즈
2022_Cus_Grade2                                                      NaN
2023_Cus_Grade1                                                      NaN
2023_Cus_Grade2                                                      NaN
2022 상반기_고객등급                                                        브론즈
2022 하반기_고객등급                                      

In [84]:
# 'Month' 칼럼을 기준으로 그룹화하고 각 그룹의 크기(행 수) 계산
month_df = df1.groupby('Month').size().reset_index(name='Count')

del month_df['Count']

# 결과 출력
print(month_df)

      Month
0   2022-01
1   2022-02
2   2022-03
3   2022-04
4   2022-05
5   2022-06
6   2022-07
7   2022-08
8   2022-09
9   2022-10
10  2022-11
11  2022-12
12  2023-01
13  2023-02
14  2023-03
15  2023-04
16  2023-05
17  2023-06
18  2023-07
19  2023-08
20  2023-09
21  2023-10
22  2023-11
23  2023-12


In [93]:
df1['ItemNameLarge'].unique()

array(['음료', '식품', '과자류', '즉석조리', '간편식사', '아이스크림'], dtype=object)

In [100]:
# 'Month'와 'ItemNameLarge'를 기준으로 데이터를 그룹화하고, 각 그룹의 크기를 계산합니다.
result = df1.groupby(['Month', 'ItemNameLarge']).size().reset_index(name='판매개수')

# 결과를 pivot_table을 사용하여 월별로 'ItemNameLarge'의 판매 개수를 나타내는 데이터 프레임으로 변환
pivot_df = result.pivot_table(index='Month', columns='ItemNameLarge', values='판매개수', fill_value=0)

pivot_df=pivot_df.reset_index()

# 결과 출력
print(pivot_df.columns)

Index(['Month', '간편식사', '과자류', '식품', '아이스크림', '음료', '즉석조리'], dtype='object', name='ItemNameLarge')


In [102]:
# 'Month'와 'ItemNameLarge'를 기준으로 데이터를 그룹화하고, 각 그룹의 크기를 계산합니다.
result2 = df1.groupby(['Month', 'ItemNameMiddle']).size().reset_index(name='판매개수')

# 결과를 pivot_table을 사용하여 월별로 'ItemNameLarge'의 판매 개수를 나타내는 데이터 프레임으로 변환
pivot_df2 = result2.pivot_table(index='Month', columns='ItemNameMiddle', values='판매개수', fill_value=0)

pivot_df2=pivot_df2.reset_index()

# 결과 출력
print(pivot_df2.columns)

Index(['Month', 'PB음료', '가공식', '껌/초콜릿', '도시락', '베이커리', '빵/디저트', '샌드위치', '스낵',
       '식재료', '아이스크림', '안주류', '유제품', '음료', '주먹밥', '즉석커피', '커피', '튀김류'],
      dtype='object', name='ItemNameMiddle')


In [131]:
discomfort_mapping = {
    '낮음': 1,
    '보통': 2,
    '높음': 3,
    '매우 높음': 4
}

df1['Discomfort_Score'] = df1['Discomfort_Grade'].map(discomfort_mapping)

# 'Month' 별로 'Discomfort_Score'의 평균값 계산
monthly_discomfort_avg = df1.groupby('Month')['Discomfort_Score'].mean().reset_index(name='Discomfort_Avg')
month_temp_avg = df1.groupby('Month')['TempAvg'].mean().reset_index(name='TempAvg')
month_temp_var = df1.groupby('Month')['Temp_Var'].mean().reset_index(name='Temp_Var')
month_humid_avg = df1.groupby('Month')['HumidAvg'].mean().reset_index(name='HumidAvg')
# 결과 확인
print(month_humid_avg)

      Month   HumidAvg
0   2022-01  55.344780
1   2022-02  55.184743
2   2022-03  61.569481
3   2022-04  54.863347
4   2022-05  55.318765
5   2022-06  73.892342
6   2022-07  77.183336
7   2022-08  80.708606
8   2022-09  67.964760
9   2022-10  68.626318
10  2022-11  64.681024
11  2022-12  59.374681
12  2023-01  62.879705
13  2023-02  58.678656
14  2023-03  50.573452
15  2023-04  61.004434
16  2023-05  61.706589
17  2023-06  73.237898
18  2023-07  80.790174
19  2023-08  76.476964
20  2023-09  73.236832
21  2023-10  66.496431
22  2023-11  64.955735
23  2023-12  68.095921


In [123]:
weather_df = pd.read_csv('./cvs_weather.csv',encoding='euc-kr')
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AreaId           790 non-null    int64  
 1   Area             790 non-null    object 
 2   dateObs          790 non-null    object 
 3   TempAvg          790 non-null    float64
 4   TempMin          789 non-null    float64
 5   hhmiTempMin      789 non-null    float64
 6   TempMax          790 non-null    float64
 7   hhmiTempMax      790 non-null    int64  
 8   hourRainfall     322 non-null    float64
 9   RainfallAmount   323 non-null    float64
 10  HumidAvg         790 non-null    float64
 11  SnowloadMax      65 non-null     float64
 12  hhmiSnowloadMax  65 non-null     float64
dtypes: float64(9), int64(2), object(2)
memory usage: 80.4+ KB


In [124]:
weather_df['RainfallAmount'] = weather_df['RainfallAmount'].fillna(0)
weather_df['hourRainfall'] = weather_df['hourRainfall'].fillna(0)

In [126]:
# 'dateObs' 칼럼을 datetime 타입으로 변환
weather_df['dateObs'] = pd.to_datetime(weather_df['dateObs'])

# 'YearMonth' 칼럼 생성 (연도와 월만 추출)
weather_df['YearMonth'] = weather_df['dateObs'].dt.to_period('M')

# 'YearMonth' 기준으로 그룹화하고 'RainfallAmount'의 합계 계산
monthly_rainfall = weather_df.groupby('YearMonth')['RainfallAmount'].sum().reset_index(name='TotalRainfall')

# 결과 확인
print(monthly_rainfall)

   YearMonth  TotalRainfall
0    2022-01            5.5
1    2022-02            4.7
2    2022-03          102.6
3    2022-04           20.4
4    2022-05            7.5
5    2022-06          393.8
6    2022-07          252.3
7    2022-08          564.8
8    2022-09          201.5
9    2022-10          124.1
10   2022-11           84.5
11   2022-12           13.6
12   2023-01           47.9
13   2023-02            1.0
14   2023-03           10.5
15   2023-04           96.9
16   2023-05          155.6
17   2023-06          195.6
18   2023-07          459.9
19   2023-08          298.1
20   2023-09          134.5
21   2023-10           31.0
22   2023-11           81.9
23   2023-12           85.9
24   2024-01           18.9
25   2024-02           74.7


In [127]:
# 'dateObs' 칼럼을 datetime 타입으로 변환
weather_df['dateObs'] = pd.to_datetime(weather_df['dateObs'])

# 'YearMonth' 칼럼 생성 (연도와 월만 추출)
weather_df['YearMonth'] = weather_df['dateObs'].dt.to_period('M')

# 'YearMonth' 기준으로 그룹화하고 'RainfallAmount'의 합계 계산
monthly_rainfall2 = weather_df.groupby('YearMonth')['hourRainfall'].sum().reset_index(name='TotalhourRainfall')

# 결과 확인
print(monthly_rainfall2)

   YearMonth  TotalhourRainfall
0    2022-01              42.43
1    2022-02              28.75
2    2022-03              72.32
3    2022-04              41.41
4    2022-05              21.00
5    2022-06             137.15
6    2022-07             101.81
7    2022-08             189.42
8    2022-09              62.93
9    2022-10              68.82
10   2022-11              35.82
11   2022-12              56.74
12   2023-01             104.80
13   2023-02              13.51
14   2023-03              15.16
15   2023-04             108.40
16   2023-05              95.17
17   2023-06              90.68
18   2023-07             156.35
19   2023-08             151.18
20   2023-09             115.49
21   2023-10              24.50
22   2023-11              72.41
23   2023-12             141.71
24   2024-01              75.42
25   2024-02             150.57


In [133]:
# 'dateObs' 칼럼을 datetime 타입으로 변환
weather_df['dateObs'] = pd.to_datetime(weather_df['dateObs'])

# 'YearMonth' 칼럼 생성 (연도와 월만 추출)
weather_df['YearMonth'] = weather_df['dateObs'].dt.to_period('M')

# 'YearMonth' 기준으로 그룹화하고 'RainfallAmount'의 합계 계산
monthly_humid = weather_df.groupby('YearMonth')['HumidAvg'].mean().reset_index(name='HumidAvg')

# 결과 확인
print(monthly_humid)

   YearMonth   HumidAvg
0    2022-01  55.212903
1    2022-02  54.696429
2    2022-03  61.932258
3    2022-04  55.030000
4    2022-05  55.335484
5    2022-06  73.376667
6    2022-07  77.232258
7    2022-08  80.170968
8    2022-09  68.090000
9    2022-10  68.887097
10   2022-11  64.390000
11   2022-12  58.919355
12   2023-01  63.303226
13   2023-02  58.900000
14   2023-03  51.241935
15   2023-04  59.933333
16   2023-05  62.493548
17   2023-06  72.433333
18   2023-07  81.074194
19   2023-08  76.351613
20   2023-09  73.483333
21   2023-10  66.477419
22   2023-11  64.503333
23   2023-12  69.348387
24   2024-01  66.561290
25   2024-02  72.948276


In [134]:
for i in range(len(month_df)):
    month_value = month_df.iloc[i]['Month']
    month_df.loc[i,'월별남자고객수'] = df1[df1['Month']==month_value]['월별남자고객수'].values[0]
    month_df.loc[i,'월별여자고객수'] = df1[df1['Month']==month_value]['월별여자고객수'].values[0]
    month_df.loc[i,'Discomfort_Avg'] = monthly_discomfort_avg[monthly_discomfort_avg['Month']==month_value]['Discomfort_Avg'].values[0]
    month_df.loc[i,'TempAvg'] = month_temp_avg[month_temp_avg['Month']==month_value]['TempAvg'].values[0]
    month_df.loc[i,'Temp_Var'] = month_temp_var[month_temp_var['Month']==month_value]['Temp_Var'].values[0]
    month_df.loc[i,'TotalRainfall'] = monthly_rainfall[monthly_rainfall['YearMonth']==month_value]['TotalRainfall'].values[0]
    month_df.loc[i,'TotalhourRainfall'] = monthly_rainfall2[monthly_rainfall2['YearMonth']==month_value]['TotalhourRainfall'].values[0]
    month_df.loc[i,'HumidAvg'] = monthly_humid[monthly_humid['YearMonth']==month_value]['HumidAvg'].values[0]
    month_df.loc[i,'PayUnit_Equal_Month'] = df1[df1['Month']==month_value]['PayUnit_Equal_Month'].values[0]
    month_df.loc[i,'(대)간편식사_판매량'] = pivot_df[pivot_df['Month']==month_value]['간편식사'].values[0]
    month_df.loc[i,'(대)과자류_판매량'] = pivot_df[pivot_df['Month']==month_value]['과자류'].values[0]
    month_df.loc[i,'(대)식품_판매량'] = pivot_df[pivot_df['Month']==month_value]['식품'].values[0]
    month_df.loc[i,'(대)아이스크림_판매량'] = pivot_df[pivot_df['Month']==month_value]['아이스크림'].values[0]
    month_df.loc[i,'(대)음료_판매량'] = pivot_df[pivot_df['Month']==month_value]['음료'].values[0]
    month_df.loc[i,'(대)즉석조리_판매량'] = pivot_df[pivot_df['Month']==month_value]['즉석조리'].values[0]
    month_df.loc[i,'(중)PB음료_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['PB음료'].values[0]
    month_df.loc[i,'(중)가공식_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['가공식'].values[0]
    month_df.loc[i,'(중)껌/초콜릿_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['껌/초콜릿'].values[0]
    month_df.loc[i,'(중)도시락_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['도시락'].values[0]
    month_df.loc[i,'(중)베이커리_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['베이커리'].values[0]
    month_df.loc[i,'(중)빵/디저트_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['빵/디저트'].values[0]
    month_df.loc[i,'(중)샌드위치_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['샌드위치'].values[0]
    month_df.loc[i,'(중)스낵_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['스낵'].values[0]
    month_df.loc[i,'(중)식재료_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['식재료'].values[0]
    month_df.loc[i,'(중)아이스크림_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['아이스크림'].values[0]
    month_df.loc[i,'(중)안주류_판매량'] = pivot_df2[pivot_df2['Month']==month_value]['안주류'].values[0]
    month_df.loc[i,'월별_매출액'] = df1[df1['Month']==month_value]['Sales_Month'].values[0]
    
# 결과 출력
print(month_df)

      Month  월별남자고객수  월별여자고객수  PayUnit_Equal_Month  간편식사 판매량  과자류 판매량  식품 판매량  \
0   2022-01   4410.0   1760.0            62328.839    1156.0   4219.0  5497.0   
1   2022-02   3432.0   1430.0            50497.221     928.0   3241.0  4245.0   
2   2022-03   5277.0   2138.0            73401.980    1392.0   4843.0  6374.0   
3   2022-04   5623.0   2265.0            80152.760    1440.0   4993.0  7100.0   
4   2022-05   6387.0   2584.0            90379.095    1612.0   5608.0  8110.0   
5   2022-06   6630.0   2648.0            93645.970    1733.0   5796.0  8320.0   
6   2022-07   6818.0   2744.0            95609.481    1759.0   6087.0  8618.0   
7   2022-08   7493.0   3023.0           107543.306    1878.0   6620.0  9467.0   
8   2022-09   7069.0   2880.0           102877.847    1867.0   6347.0  8881.0   
9   2022-10   7640.0   3032.0           106315.165    1924.0   6689.0  9463.0   
10  2022-11   6820.0   2750.0            94425.056    1818.0   6251.0  8443.0   
11  2022-12   5968.0   2357.